# VOQC Tutorial (PLanQC 2020)

We provide Python bindings for VOQC in [inQWIRE/pyvoqc](https://github.com/inQWIRE/pyvoqc). Providing Python bindings makes it easier to perform evaluations and easier to integrate our tool into existing toolchains.

## Preliminaries

To run this tutorial: 
1. Install our OCaml package with `opam install voqc` (requires opam)
2. Run `./install.sh` in the pyvoqc directory

## Using PyVOQC Directly

In [4]:
from pyvoqc.voqc import VOQC

# load circuit
c = VOQC("tutorial-files/tof_3_example.qasm")
print("Input file:")
c.print_info()

# decompose CCX gates into single-qubit and CX (= cnot) gates
c.decompose_to_cnot()
print("\nAfter decomposing CCX gates:")
c.print_info()

# run our most general optimization (see Sec. 4 of our POPL 2021 paper)
c.optimize_nam()
print("\nAfter optimization:")
c.print_info()

# Rzq is a non-standard gate; replace it with Rz for compatability
c.replace_rzq()
print("\nAfter removing Rzq gates:")
c.print_info()

# map the circuit to the Tenerife architecture with initial layout [0,1,2,3,4]
c.make_tenerife()
c.list_to_layout([0,1,2,3,4])
c.simple_map()
print("\nAfter mapping:")
c.print_info() # adds a bunch of CX and H gates!

# try optimizing again to remove introduced gates
c.optimize_nam()
print("\nAfter optimization (round 2):")
c.print_info()

# write the mapped and optimized circuit
c.write("out.qasm")

Input file:
Circuit uses 5 qubits and 3 gates.
{'CCX': 3}

After decomposing CCX gates:
Circuit uses 5 qubits and 45 gates.
{'H': 6, 'T': 12, 'Tdg': 9, 'CX': 18}

After optimization:
Circuit uses 5 qubits and 40 gates.
{'H': 6, 'Rzq': 18, 'CX': 16}

After removing Rzq gates:
Circuit uses 5 qubits and 40 gates.
{'H': 6, 'S': 2, 'T': 8, 'Sdg': 1, 'Tdg': 7, 'CX': 16}

After mapping:
Circuit uses 5 qubits and 159 gates.
{'H': 98, 'S': 2, 'T': 8, 'Sdg': 1, 'Tdg': 7, 'CX': 43}
Current layout is [2,1,0,3,4]

After optimization (round 2):
Circuit uses 5 qubits and 57 gates.
{'H': 6, 'Rzq': 18, 'CX': 33}
Current layout is [2,1,0,3,4]


## Running VOQC as a Qiskit Pass

Using our "voqc_pass" wrapper, VOQC can be called just like any other optimization pass in [IBM's Qiskit framework](https://qiskit.org/documentation/getting_started.html). This allows us to take advantage of Qiskit's utilities for quantum programming, such as the ability to build and print circuits.

To use VOQC, simply append `QiskitVOQC([opt list])` to a Qiskit `Pass Manager` where `opt list` is an optional argument specifying one or more of the transformations in VOQC. `QiskitVOQC()` with no arguments will run all optimizations available.

In [11]:
from qiskit import QuantumCircuit
from pyvoqc.qiskit.voqc_pass import QiskitVOQC
from qiskit.transpiler import PassManager

# create the circuit above
circ = QuantumCircuit(2)
circ.x(0)
circ.t(0)
circ.t(1)
circ.cz(0, 1)
circ.t(0)
circ.tdg(1)
print("Before Optimization:")
print(circ)

# create a Qiskit PassManager
pm = PassManager()

# decompose CZ gate
pm.append(QiskitVOQC(["decompose_to_cnot"]))
new_circ = pm.run(circ)
print("\n\nAfter 'decompose_to_cnot':")
print(new_circ)

# run optimizations from Nam et al.
pm.append(QiskitVOQC(["optimize_nam", "replace_rzq"]))
new_circ = pm.run(circ)
print("\n\nAfter 'optimize_nam':")
print(new_circ)

# run IBM gate merging
pm.append(QiskitVOQC(["optimize_ibm"]))
new_circ = pm.run(circ)
print("\n\nAfter 'optimize_ibm':")
print(new_circ)


Before Optimization:
     ┌───┐┌───┐    ┌───┐ 
q_0: ┤ X ├┤ T ├─■──┤ T ├─
     ├───┤└───┘ │ ┌┴───┴┐
q_1: ┤ T ├──────■─┤ TDG ├
     └───┘        └─────┘


After 'decompose_to_cnot':
     ┌───┐┌───┐     ┌───┐       
q_0: ┤ X ├┤ T ├──■──┤ T ├───────
     ├───┤├───┤┌─┴─┐├───┤┌─────┐
q_1: ┤ T ├┤ H ├┤ X ├┤ H ├┤ TDG ├
     └───┘└───┘└───┘└───┘└─────┘


After 'optimize_nam':
               ┌─────┐┌───┐
q_0: ───────■──┤ SDG ├┤ X ├
     ┌───┐┌─┴─┐└┬───┬┘├───┤
q_1: ┤ H ├┤ X ├─┤ H ├─┤ Z ├
     └───┘└───┘ └───┘ └───┘


After 'optimize_ibm':
                      ┌───────────────┐
q_0: ──────────────■──┤ U3(pi,0,pi/2) ├
     ┌──────────┐┌─┴─┐└─┬───────────┬─┘
q_1: ┤ U2(0,pi) ├┤ X ├──┤ U2(pi,pi) ├──
     └──────────┘└───┘  └───────────┘  
